# GAAP Features Report July 2023 
**This report details GAAP data collection efforts through 07/17/2023.** 

## Executive Summary

The GAAP Features source code library processes all raw SEC 10-Q filing document collections extracting the filing's XBRL instance document and all XBRL facts disclosed by the filer. The XBRL instance document typically references multiple taxonomies such as the US-GAAP taxonomy that define the numerous individual XBRL facts contained within the instance document.  Each fact typically contains one piece of digital data (the value) and all of the associated metadata (name, data type, decimal places, debit / credit, USD, shares, ratio, text block, labels, taxonomy url, namespace etc) that define the fact's value. One fact can contain any number of metadata elements defined in its taxonomy definition.  One filing may reference the US-GAAP taxonomy and any other number of other taxonomies containing fact specific definitions.

SEC filings use XBRL facts to disclose filing metadata, report text blocks, fiscal duration data, fiscal instance data, share data, ratio data, filing flags, and other filing specific information. The GAAP Features library processes XBRL facts and uses the filing specific metadata to organize and create filing and fact specific views for all of the data disclosed.

Fiscal instance and duration data contain all financial assets, income, cash flows, and balance sheet data included within each filing. The library uses both instance and duration data to identify common accounting periods and financial disclosures contained within 10-Q filings such as: quarter over quarter, year over year, YTD over YTD, document / fiscal period ending, and fiscal year ending financial performance results.  

Once fiscal instance and duration records are categorized and matched, percentage changes may also be determined for financial performance elements disclosed across multiple fiscal instances or durations.  For example, two fiscal durations make up the current quarterly results (duration 1) for a filer and the same period for the prior year (duration 2).  Percentage changes for any XRBL fact disclosed in both fiscal durations, such as gross income, can be determined.  

The current library calculates percentage changes for all financial items disclosed in Year over Year quarterly durations (QoQ), fiscal YTD over YTD durations (YoY), and the fiscal year end instant over document period ending instant (FYED to DPED).    

## Library Functionality and Coverages
The remainder of this report shows detailed examples and explanations for each data view extracted from a sample filing.  In addition, coverages are provided for each type of filing view collected.  

**Total Filings collected since 2019** 
* Count: 81953

**XBRL Instance Documents Extracted** 
* Count: 79733 
* Coverage: 97.29% 
* Most (if not all) of the missing files are from filings that did not submit an XBRL instance document. 

**GAAP Feature Files** 
* Count: 79733 
* Coverage: 97.29% 

**GAAP Metadata**  
* Count: 79733 
* Coverage: 97.29% 

**Filings with Fiscal Durations**
* Count: 79733 
* Coverage: 97.29% 

**Filings with >=2 Fiscal Durations**
* Count: 79341 
* Coverage: 96.81% 

**Filings with Fiscal Instances** 
* Count: 79733 
* Coverage: 97.29% 

**Filings with >=2 Fiscal Instances** 
* Count: 79522 
* Coverage: 97.03% 

**Filings with Matched Year over Year Quarterly Durations (QoQ)** 
* Count: 76367 
* Coverage: 93.18% 

**Filings with Matched Fiscal YTD over YTD Durations (YoY)** 
* Count: 75655 
* Coverage: 92.32% 

**Filings with Matched Fiscal Year End Over Document Period End Instances (FYED to DPED)**
* Count: 78605 
* Coverage: 95.91% 

**Filings with Year over Year Quarterly Percentage Features (QoQ)**  
* Count: 76367 
* Coverage: 93.18%

**Filings with Fiscal YTD over YTD Percentage Features (YoY)** 
* Count: 75655 
* Coverage: 92.32% 

**Filings with Matched Fiscal Year End Over Document Period End Percentage Features (FYED to DPED)** 
* Count: 78605 
* Coverage: 95.91% 

**Filings with a Combined GAAP Percentage Features File**  
* Count: 79733
* Coverage: 97.29% 



## Report Sections 
The remainder of this report shows detailed examples of the data extracted from a sample filing.  In addition, coverage explantions are provided for each type of filing data collected.    

1. GAAP Source Code Library Overview
2. Filing XBRL Overview
3. Fiscal Instant Data
4. Fiscal Duration Data
5. Filing Metadata and Text Blocks
6. Share Data
7. Ratio Data
8. GAAP Data Collection and Coverages
9. XBRL instance files
10. Raw GAAP Fact / Feature Files
11. GAAP Features Library
12. GAAP Feature Coverages
13. Filing Metadata View
14. Fiscal Durations View
15. Matched Fiscal Duration Views
16. Percentage Change Duration View
17. Fiscal Instances View
18. Matched Fiscal Instance Views
19. Percentage Change Instance Views
20. GAAP Percentage Features View
21. GAAP Full Data Repository

# GAAP Source Code Library Overview 
The GAAP source code library comprises functions and data repositories to collect and maintain all filing metadata and financials extracted from the XBRL facts in SEC 10-Q Filings.

* **XBRLInstance.py** - 193 lines
 * Get all latest taxonomies from the sec, updates the Py-XBRL parser
 * Finds any filings that do not have GAAP features extracted. 
 * Searches XBRL document collections. 
 * Identifies and extracts the XBRL instance document. 
 
* **GaapExtractor.py** - 102 lines (Recursive serialization to flatten XBRL fact metadata **possibly patentable**) 
 *  Recursively serializes all nested objects in each XBRL fact to collect all fact specific metadata provided by the filing's taxonomy.
 * Provides all filing facts, fact values, and fact metadata as a csv file.
 
* **GaapFeatures.py** - 918 lines (identification of QoQ, YoY, FYED_to_DPED and conversions to percentage changes **possibly patentable**)
 * Matches fiscal instance and duration periods within each filing. 
 * Creates all GAAP Feature views available for each filing.

* **GaapDataRepo.py** - 199 lines 
 * Creates the GAAP Full Data Repository dataframe, sparse feature matrix, and feature names index. 
 * GAAP_metadata_10Q_Full
 * GAAP_Pct_Chg_10Q_Full
 * GAAP_Pct_Chg_10Q_Full_Feature_Names

# Filing XBRL Overview
* Each compliant filing's XBRL document collection contains an XBRL instance document.
* XBRL instance documents contain all of the digital data disclosed by each company. 
* Individual data elements are stored in the XBRL instance document as XBRL Facts.  
* XBRL Facts are defined within taxonomies that contain all of the metadata relevant to each individual fact.
* The most common taxonomy for sec filings is the US-GAAP taxonomy. 
* However, a single filing typically contains many different taxonomies that define many different facts. 

### Example Filing - XBRL Facts and Associated Metadata
* The following list shows the XBRL Facts and metadata extracted for a single example filing.
* This filing has 41 columns of data related to the 648 XBRL facts disclosed. 
* The concept_name and value disclose the individual XBRL fact names and associated values. 

In [20]:
import pandas as pd
import Source.MonitorFilings.Parameters as params
from Source.GAAP.GaapFeatures import * 


file_name = '726865_0000726865-21-000388.csv'

date_cols = ['context_start_date','context_end_date','context_instant_date']
GAAP_csv = pd.read_csv(params.GAAP_dir + file_name, parse_dates=date_cols)

print('\n\nXBRL Facts and Associated Metadata Available\n\n')

GAAP_csv.info()



XBRL Facts and Associated Metadata Available


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648 entries, 0 to 647
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   concept_xml_id               648 non-null    object        
 1   concept_schema_url           648 non-null    object        
 2   concept_name                 648 non-null    object        
 3   concept_substitution_group   648 non-null    object        
 4   concept_concept_type         0 non-null      float64       
 5   concept_abstract             648 non-null    bool          
 6   concept_nillable             648 non-null    bool          
 7   concept_period_type          648 non-null    object        
 8   concept_balance              529 non-null    object        
 9   concept_labels_0_label       648 non-null    object        
 10  concept_labels_0_language    648 non-null    object        
 

### XBRL Facts and Associated Metadata Sample
* Each XBRL fact contains the taxonomy, schema, and id that defines the fact.  
* It also contains detailed metadata about the fact that informs the user exactly what type of digital data is contained within each fact.

In [33]:
display_full_dataframe(GAAP_csv.tail(25))

,concept_xml_id,concept_schema_url,concept_name,concept_substitution_group,concept_concept_type,concept_abstract,concept_nillable,concept_period_type,concept_balance,concept_labels_0_label,concept_labels_0_language,concept_labels_0_text,concept_labels_0_label_type,concept_type,context_xml_id,context_entity,context_start_date,context_end_date,value,xml_id,footnote,context_instant_date,unit_unit_id,unit_unit,decimals,concept_labels_1_label,concept_labels_1_language,concept_labels_1_text,concept_labels_1_label_type,concept_labels_2_label,concept_labels_2_language,concept_labels_2_text,concept_labels_2_label_type,concept_labels_3_label,concept_labels_3_language,concept_labels_3_text,concept_labels_3_label_type,concept_labels_4_label,concept_labels_4_language,concept_labels_4_text,concept_labels_4_label_type
623,us-gaap_FairValueAssetRecurringBasisStillHeldU...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...,FairValueAssetRecurringBasisStillHeldUnrealize...,item,NaN,False,True,duration,credit,us-gaap_FairValueAssetRecurringBasisStillHeldU...,en-US,"Fair Value, Asset, Recurring Basis, Still Held...",http://www.xbrl.org/2003/role/label,xbrli:monetaryItemType,Duration_1_1_2020_To_6_30_2020,726865,2020-01-01,2020-06-30,136000000.0,ct-nonFraction-832bfd4f-f933-448b-bb0b-b04a9d3...,NaN,NaT,Unit1,iso4217:USD,-6.0,us-gaap_FairValueAssetRecurringBasisStillHeldU...,en-US,"Unrealized gains (losses) included in OCI, net",http://www.xbrl.org/2003/role/verboseLabel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
624,cik0000726865_FairValueAssetsAndLiabilitiesTra...,https://www.sec.gov/Archives/edgar/data/726865...,FairValueAssetsAndLiabilitiesTransfersInAndOut...,item,NaN,False,True,duration,NaN,cik0000726865_FairValueAssetsAndLiabilitiesTra...,en-US,Presentation in tabular form of the components...,http://www.xbrl.org/2003/role/documentation,nonnum:textBlockItemType,Duration_1_1_2021_To_6_30_2021,726865,2021-01-01,2021-06-30,"<p dir=""ltr"" style=""font-family: 'Garamond', '...",ct-nonNumeric-b8043685-a2bf-45ea-9f3c-8e84091b...,NaN,NaT,NaN,NaN,NaN,cik0000726865_FairValueAssetsAndLiabilitiesTra...,en-US,Fair Value Assets And Liabilities Transfers In...,http://www.xbrl.org/2003/role/label,cik0000726865_FairValueAssetsAndLiabilitiesTra...,en-US,Components Of The Transfers In And Out Of Level 3,http://www.xbrl.org/2003/role/terseLabel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
625,us-gaap_FairValueMeasurementWithUnobservableIn...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...,FairValueMeasurementWithUnobservableInputsReco...,item,NaN,False,True,duration,credit,us-gaap_FairValueMeasurementWithUnobservableIn...,en-US,Fair Value Measurement With Unobservable Input...,http://www.xbrl.org/2003/role/label,xbrli:monetaryItemType,Duration_4_1_2021_To_6_30_2021,726865,2021-04-01,2021-06-30,-762000000.0,ct-nonFraction-6843d083-7d50-40f9-9d25-25db7c1...,NaN,NaT,Unit1,iso4217:USD,-6.0,us-gaap_FairValueMeasurementWithUnobservableIn...,en-US,Transfers Out of Level 3,http://www.xbrl.org/2009/role/negatedLabel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
626,us-gaap_FairValueMeasurementWithUnobservableIn...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...,FairValueMeasurementWithUnobservableInputsReco...,item,NaN,False,True,duration,debit,us-gaap_FairValueMeasurementWithUnobservableIn...,en-US,Fair Value Measurement With Unobservable Input...,http://www.xbrl.org/2003/role/label,xbrli:monetaryItemType,Duration_4_1_2021_To_6_30_2021,726865,2021-04-01,2021-06-30,762000000.0,ct-nonFraction-b753074b-88d6-4187-b262-59b6a3a...,NaN,NaT,Unit1,iso4217:USD,-6.0,us-gaap_FairValueMeasurementWithUnobservableIn...,en-US,Total,http://www.xbrl.org/2003/role/totalLabel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
627,us-gaap_FairValueMeasurementWithUnobservableIn...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...,FairValueMeasurementWithUnobservableInputsReco...,item,NaN,False,True,duration,debit,us-gaap_FairValueMeasurementWithUnobservableIn...,en-US,Fair Value Measurement With

#### --------------------------------------------------------------------------------------------------------------------------------------------------------------------------

**NOTE: All remaining data and GAAP feature views shown below are generated from the XBRL facts and metadata extracted from the XBRL Instance Document.**  
#### --------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Fiscal Instance Data

This data is a snapshot of the company's financial position at a specific point in time, usually at the end of a fiscal period. This can include information like assets, liabilities, and shareholders' equity at the fiscal year-end. This data allows investors to understand the company's current financial position, its assets and liabilities, and any changes since the end of the last fiscal year.

In SEC filings, Fiscal Instant Data represents a snapshot of each fact as of the context_instant_date. A balance sheet is a snapshot of a company's financial situation at a specific point in time, so the "context_instant_date" values represent when each of these snapshots was taken. 

Some of the instances disclosed in a filing may simply correspond to less extensive updates, while others correspond to full-blown financial reports with comprehensive disclosure requirements. The level of detail on particular fiscal instance dates is a reflection of the nature of the disclosures required at the end of key financial reporting periods. Adjustments may only contain only a few key XBRL facts, while instances for a fiscal period may contain an entire financial report.   

### Typical 10-Q Filing Instances

Most 10-Q filings disclose and compare at least two fiscal instances. 

* **Fiscal Year-End Date (FYED) to Document Period End Date (DPED):** Compares the balance sheet data (which provides a snapshot of the company's financial position at a specific point in time) at the end of the fiscal year with the balance sheet data at the end of the reporting period for the 10-Q filing.

In addition, many companies will also disclose instant data as of the last fiscal year end and sometimes the prior fiscal year end.  

### Fiscal Instant Data - Debits
* In SEC filings, Fiscal Instant Data represents a snapshot of each fact as of the context_instant_date. 
* The concept_balance also informs what type of balance sheet item we are looking at: (debit / credit / null). 
* In the example filing below, we can see:
 * All balance sheet debit snapshots and the date the snapshot was taken. 
 * All items appear to be in US dollars and captured at -6 decimals.
 * Inspecting the concept_schema_url also informs us that most of these debit items are defined in the US-GAAP taxonomy.
 * However, some items are derived from a custom taxonomy that is specific to the filer's CIK number and stored at the SEC. 
 * Some debits on the balance sheet are disclosed on up to 6 different instance dates.

In [39]:
df_cols = ['concept_name','context_instant_date','value','concept_balance','concept_period_type','unit_unit','decimals', 
          'concept_xml_id', 'concept_schema_url']

df = GAAP_csv[df_cols][(GAAP_csv.concept_balance == 'debit') & 
                       (GAAP_csv.concept_period_type == 'instant')
                      ].sort_values(['concept_name','context_instant_date'])

display_full_dataframe(df)

,concept_name,context_instant_date,value,concept_balance,concept_period_type,unit_unit,decimals,concept_xml_id,concept_schema_url
55,AccruedInvestmentIncomeReceivable,2020-12-31,1217000000.0,debit,instant,iso4217:USD,-6.0,us-gaap_AccruedInvestmentIncomeReceivable,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
54,AccruedInvestmentIncomeReceivable,2021-06-30,1215000000.0,debit,instant,iso4217:USD,-6.0,us-gaap_AccruedInvestmentIncomeReceivable,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
67,Assets,2020-12-31,368319000000.0,debit,instant,iso4217:USD,-6.0,us-gaap_Assets,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
66,Assets,2021-06-30,384128000000.0,debit,instant,iso4217:USD,-6.0,us-gaap_Assets,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
27,AvailableForSaleDebtSecuritiesAmortizedCostBasis,2020-12-31,103021000000.0,debit,instant,iso4217:USD,-6.0,us-gaap_AvailableForSaleDebtSecuritiesAmortize...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
26,AvailableForSaleDebtSecuritiesAmortizedCostBasis,2021-06-30,105530000000.0,debit,instant,iso4217:USD,-6.0,us-gaap_AvailableForSaleDebtSecuritiesAmortize...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
31,AvailableForSaleSecuritiesDebtSecurities,2020-12-31,121111000000.0,debit,instant,iso4217:USD,-6.0,us-gaap_AvailableForSaleSecuritiesDebtSecurities,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
30,AvailableForSaleSecuritiesDebtSecurities,2021-06-30,120452000000.0,debit,instant,iso4217:USD,-6.0,us-gaap_AvailableForSaleSecuritiesDebtSecurities,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
253,CashCashEquivalentsRestrictedCashAndRestricted...,2019-12-31,1879000000.0,debit,instant,iso4217:USD,-6.0,us-gaap_CashCashEquivalentsRestrictedCashAndRe...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
255,CashCashEquivalentsRestrictedCashAndRestricted...,2020-06-30,3219000000.0,debit,instant,iso4217:USD,-6.0,us-gaap_CashCashEquivalentsRestrictedCashAndRe...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...


### Fiscal Instant Data - Credits
* In SEC filings, Fiscal Instant Data represents a snapshot of each fact as of the context_instant_date. 
* The concept_balance also informs what type of balance sheet item we are looking at: (debit / credit / null). 
* In the example filing below, we can see:
 * All balance sheet credit snapshots and the date the snapshot was taken. 
 * All items appear to be in US dollars and captured at -6 decimals.
 * Inspecting the concept_schema_url also informs us that most of these debit items are defined in the US-GAAP taxonomy.
 * However, some items are derived from a custom taxonomy that is specific to the filer's CIK number and stored at the SEC. 
 * Some credits on the balance sheet are disclosed up to 6 times.
 * While StockholdersEquity (last entry) is disclosed 6 different times, some of the instant dates and values are identical which indicates that the same data is likely displayed in multiple locations on this filing.   

In [40]:
df_cols = ['concept_name','context_instant_date','value','concept_balance','concept_period_type','unit_unit','decimals', 
          'concept_xml_id', 'concept_schema_url']

df = GAAP_csv[df_cols][(GAAP_csv.concept_balance == 'credit') & 
                       (GAAP_csv.concept_period_type == 'instant')
                      ].sort_values(['concept_name','context_instant_date'])

display_full_dataframe(df)

,concept_name,context_instant_date,value,concept_balance,concept_period_type,unit_unit,decimals,concept_xml_id,concept_schema_url
99,AccumulatedOtherComprehensiveIncomeLossNetOfTax,2020-12-31,9021000000.0,credit,instant,iso4217:USD,-6.0,us-gaap_AccumulatedOtherComprehensiveIncomeLos...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
98,AccumulatedOtherComprehensiveIncomeLossNetOfTax,2021-06-30,7561000000.0,credit,instant,iso4217:USD,-6.0,us-gaap_AccumulatedOtherComprehensiveIncomeLos...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
330,CollateralPayableHeldForDerivativeInvestmentsC...,2020-12-31,2970000000.0,credit,instant,iso4217:USD,-6.0,cik0000726865_CollateralPayableHeldForDerivati...,https://www.sec.gov/Archives/edgar/data/726865...
328,CollateralPayableHeldForDerivativeInvestmentsC...,2021-06-30,4384000000.0,credit,instant,iso4217:USD,-6.0,cik0000726865_CollateralPayableHeldForDerivati...,https://www.sec.gov/Archives/edgar/data/726865...
95,CommonStockValueOutstanding,2020-12-31,11853000000.0,credit,instant,iso4217:USD,-6.0,us-gaap_CommonStockValueOutstanding,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
94,CommonStockValueOutstanding,2021-06-30,11930000000.0,credit,instant,iso4217:USD,-6.0,us-gaap_CommonStockValueOutstanding,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
73,DebtCurrent,2020-12-31,497000000.0,credit,instant,iso4217:USD,-6.0,us-gaap_DebtCurrent,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
72,DebtCurrent,2021-06-30,457000000.0,credit,instant,iso4217:USD,-6.0,us-gaap_DebtCurrent,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
281,DebtSecuritiesAvailableForSaleAllowanceForCred...,2020-03-31,18000000.0,credit,instant,iso4217:USD,-6.0,us-gaap_DebtSecuritiesAvailableForSaleAllowanc...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
284,DebtSecuritiesAvailableForSaleAllowanceForCred...,2020-06-30,21000000.0,credit,instant,iso4217:USD,-6.0,us-gaap_DebtSecuritiesAvailableForSaleAllowanc...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...


# Fiscal Duration Data

Fiscal duration data represent durations over which income statement and cash flow data are accumulated. Typically the fiscal durations allow investors to see how the company is doing compared to a similar period in the past and determine if it's growing, stagnating, or declining.

**Typical 10-Q Filing Durations**

Most 10-Q filings disclose two durations for the current fiscal year and compare the data in these durations to the same time period from the previous year.   

* **Year-over-Year Quarterly Fiscal Durations (QoQ):** Compares a given quarter's data with the same quarter from the previous year. For example, you could compare Q2 2023 with Q2 2022.

* **YTD over YTD (Year-to-Date) Durations (YoY):** Compares the cumulative total for a given fiscal year up until a certain date with the cumulative total for the previous year up until the same date. For example, you could compare YTD as of Q2 2023 with YTD as of Q2 2022.

**NOTE**

In a perfect world, the dates or durations for two QoQ or YoY / YTD durations would match.  However, this is certainly NOT the case in practice.  For example, a current QoQ duration may cover 90 days and the prior year may cover 93 days etc.  Start and end dates may also be off by a few days. Furthermore, matching durations may not always disclose the exact same financial data elements.   

It takes some fairly sophisticated matching logic to ensure that the proper QoQ, YoY and YTD durations are matched. 

### Fiscal Duration Data - Debits
* In SEC filings, Fiscal Duration Data represents each fact's value between the context start and end dates.
* The concept_balance also informs what type of balance sheet item we are looking at: (debit / credit / null). 
* In the example filing below, we can see:
 * All balance sheet debits and values between the context start and end dates.
 * Units and taxonomies are the same as the fiscal instant data above. 
 * In the raw dataview, many of the fiscal duration elements are disclosed across multiple durations. 

In [42]:
df_cols = ['concept_name','context_start_date','context_end_date','value','concept_balance','concept_period_type','unit_unit','decimals', 
          'concept_xml_id', 'concept_schema_url']

df = GAAP_csv[df_cols][(GAAP_csv.concept_balance == 'debit') & 
                       (GAAP_csv.concept_period_type == 'duration')
                      ].sort_values(['concept_name','context_start_date', 'context_end_date'])

display_full_dataframe(df)

,concept_name,context_start_date,context_end_date,value,concept_balance,concept_period_type,unit_unit,decimals,concept_xml_id,concept_schema_url
264,AffordableHousingTaxCreditsAndOtherTaxExpenseA...,2020-01-01,2020-06-30,-1000000.0,debit,duration,iso4217:USD,-6.0,cik0000726865_AffordableHousingTaxCreditsAndOt...,https://www.sec.gov/Archives/edgar/data/726865...
155,BenefitsLossesAndExpenses,2020-01-01,2020-06-30,7397000000.0,debit,duration,iso4217:USD,-6.0,us-gaap_BenefitsLossesAndExpenses,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
153,BenefitsLossesAndExpenses,2020-04-01,2020-06-30,3757000000.0,debit,duration,iso4217:USD,-6.0,us-gaap_BenefitsLossesAndExpenses,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
154,BenefitsLossesAndExpenses,2021-01-01,2021-06-30,8018000000.0,debit,duration,iso4217:USD,-6.0,us-gaap_BenefitsLossesAndExpenses,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
152,BenefitsLossesAndExpenses,2021-04-01,2021-06-30,3891000000.0,debit,duration,iso4217:USD,-6.0,us-gaap_BenefitsLossesAndExpenses,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
251,CashCashEquivalentsRestrictedCashAndRestricted...,2020-01-01,2020-06-30,1340000000.0,debit,duration,iso4217:USD,-6.0,us-gaap_CashCashEquivalentsRestrictedCashAndRe...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
250,CashCashEquivalentsRestrictedCashAndRestricted...,2021-01-01,2021-06-30,452000000.0,debit,duration,iso4217:USD,-6.0,us-gaap_CashCashEquivalentsRestrictedCashAndRe...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
288,DebtSecuritiesAvailableForSaleAccruedInterestW...,2020-01-01,2020-06-30,1000000000.0,debit,duration,iso4217:USD,-8.0,us-gaap_DebtSecuritiesAvailableForSaleAccruedI...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
289,DebtSecuritiesAvailableForSaleAccruedInterestW...,2021-01-01,2021-06-30,1000000000.0,debit,duration,iso4217:USD,-8.0,us-gaap_DebtSecuritiesAvailableForSaleAccruedI...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
285,DebtSecuritiesAvailableForSaleAllowanceForCred...,2020-01-01,2020-06-30,36000000.0,debit,duration,iso4217:USD,-6.0,us-gaap_DebtSecuritiesAvailableForSaleAllowanc...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...


### Fiscal Duration Data - Credits

In [43]:
df_cols = ['concept_name','context_start_date','context_end_date','value','concept_balance','concept_period_type','unit_unit','decimals', 
          'concept_xml_id', 'concept_schema_url']

df = GAAP_csv[df_cols][(GAAP_csv.concept_balance == 'credit') & 
                       (GAAP_csv.concept_period_type == 'duration')
                      ].sort_values(['concept_name','context_start_date', 'context_end_date'])

display_full_dataframe(df)

,concept_name,context_start_date,context_end_date,value,concept_balance,concept_period_type,unit_unit,decimals,concept_xml_id,concept_schema_url
263,AffordableHousingTaxCreditsAndOtherTaxBenefits...,2021-01-01,2021-06-30,2000000.0,credit,duration,iso4217:USD,-6.0,us-gaap_AffordableHousingTaxCreditsAndOtherTax...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
262,AffordableHousingTaxCreditsAndOtherTaxBenefits...,2021-04-01,2021-06-30,1000000.0,credit,duration,iso4217:USD,-6.0,us-gaap_AffordableHousingTaxCreditsAndOtherTax...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
527,AmortizationAssociatedWithRealizedGainLossOnCe...,2020-01-01,2020-06-30,43000000.0,credit,duration,iso4217:USD,-6.0,cik0000726865_AmortizationAssociatedWithRealiz...,https://www.sec.gov/Archives/edgar/data/726865...
525,AmortizationAssociatedWithRealizedGainLossOnCe...,2020-04-01,2020-06-30,18000000.0,credit,duration,iso4217:USD,-6.0,cik0000726865_AmortizationAssociatedWithRealiz...,https://www.sec.gov/Archives/edgar/data/726865...
526,AmortizationAssociatedWithRealizedGainLossOnCe...,2021-01-01,2021-06-30,-10000000.0,credit,duration,iso4217:USD,-6.0,cik0000726865_AmortizationAssociatedWithRealiz...,https://www.sec.gov/Archives/edgar/data/726865...
524,AmortizationAssociatedWithRealizedGainLossOnCe...,2021-04-01,2021-06-30,-5000000.0,credit,duration,iso4217:USD,-6.0,cik0000726865_AmortizationAssociatedWithRealiz...,https://www.sec.gov/Archives/edgar/data/726865...
123,AmortizationOfDeferredGainOnBusinessSoldThroug...,2020-01-01,2020-06-30,18000000.0,credit,duration,iso4217:USD,-6.0,cik0000726865_AmortizationOfDeferredGainOnBusi...,https://www.sec.gov/Archives/edgar/data/726865...
187,AmortizationOfDeferredGainOnBusinessSoldThroug...,2020-01-01,2020-06-30,18000000.0,credit,duration,iso4217:USD,-6.0,cik0000726865_AmortizationOfDeferredGainOnBusi...,https://www.sec.gov/Archives/edgar/data/726865...
121,AmortizationOfDeferredGainOnBusinessSoldThroug...,2020-04-01,2020-06-30,9000000.0,credit,duration,iso4217:USD,-6.0,cik0000726865_AmortizationOfDeferredGainOnBusi...,https://www.sec.gov/Archives/edgar/data/726865...
122,AmortizationOfDeferredGainOnBusinessSoldThroug...,2021-01-01,2021-06-30,14000000.0,credit,duration,iso4217:USD,-6.0,cik0000726865_AmortizationOfDeferredGainOnBusi...,https://www.sec.gov/Archives/edgar/data/726865...


# Filing Metadata and Text Blocks

**SEC filings also contain additional XBRL facts disclosing information such as:**

* **company details** - Name, Address, Country, Phone Number
* **filing specific metadata** - CurrentFiscalYearEndDate, DocumentFiscalPeriodFocus
* **accounting and report text blocks** - GainLossOnInvestmentsTextBlock, ScheduleOfRealizedGainLossTableTextBlock etc.
* **filing flags** - AmendmentFlag etc. (some filings contain many other flags)

In [46]:
df_cols = ['concept_name','context_start_date','context_end_date','value','concept_balance','concept_period_type','unit_unit','decimals', 
          'concept_xml_id', 'concept_schema_url']

df = GAAP_csv[df_cols][(GAAP_csv.concept_balance.isnull()) & (GAAP_csv.unit_unit.isnull())]

display_full_dataframe(df)

,concept_name,context_start_date,context_end_date,value,concept_balance,concept_period_type,unit_unit,decimals,concept_xml_id,concept_schema_url
0,AmendmentFlag,2021-01-01,2021-06-30,false,NaN,duration,NaN,NaN,dei_AmendmentFlag,https://xbrl.sec.gov/dei/2020/dei-2020-01-31.xsd
1,CurrentFiscalYearEndDate,2021-01-01,2021-06-30,--12-31,NaN,duration,NaN,NaN,dei_CurrentFiscalYearEndDate,https://xbrl.sec.gov/dei/2020/dei-2020-01-31.xsd
2,DocumentFiscalPeriodFocus,2021-01-01,2021-06-30,Q2,NaN,duration,NaN,NaN,dei_DocumentFiscalPeriodFocus,https://xbrl.sec.gov/dei/2020/dei-2020-01-31.xsd
3,EntityCentralIndexKey,2021-01-01,2021-06-30,0000726865,NaN,duration,NaN,NaN,dei_EntityCentralIndexKey,https://xbrl.sec.gov/dei/2020/dei-2020-01-31.xsd
4,EntityCurrentReportingStatus,2021-01-01,2021-06-30,Yes,NaN,duration,NaN,NaN,dei_EntityCurrentReportingStatus,https://xbrl.sec.gov/dei/2020/dei-2020-01-31.xsd
5,EntityInteractiveDataCurrent,2021-01-01,2021-06-30,Yes,NaN,duration,NaN,NaN,dei_EntityInteractiveDataCurrent,https://xbrl.sec.gov/dei/2020/dei-2020-01-31.xsd
6,DocumentType,2021-01-01,2021-06-30,10-Q,NaN,duration,NaN,NaN,dei_DocumentType,https://xbrl.sec.gov/dei/2020/dei-2020-01-31.xsd
7,DocumentQuarterlyReport,2021-01-01,2021-06-30,true,NaN,duration,NaN,NaN,dei_DocumentQuarterlyReport,https://xbrl.sec.gov/dei/2020/dei-2020-01-31.xsd
8,DocumentPeriodEndDate,2021-01-01,2021-06-30,2021-06-30,NaN,duration,NaN,NaN,dei_DocumentPeriodEndDate,https://xbrl.sec.gov/dei/2020/dei-2020-01-31.xsd
9,DocumentFiscalYearFocus,2021-01-01,2021-06-30,2021,NaN,duration,NaN,NaN,dei_DocumentFiscalYearFocus,https://xbrl.sec.gov/dei/2020/dei-2020-01-31.xsd


# Share Data
* Some XBRL facts are disclosed in the form of shares.
* All the share data disclosed in this example filing is fiscal instant data meaning that it was disclosed as a snapshot on a particular instant date. 

In [52]:
df_cols = ['concept_name','context_instant_date','value','concept_balance','concept_period_type','unit_unit','decimals', 
          'concept_xml_id', 'concept_schema_url']

df = GAAP_csv[df_cols][(GAAP_csv.unit_unit == 'shares')].sort_values(['concept_name','context_instant_date'])

display_full_dataframe(df)

,concept_name,context_instant_date,value,concept_balance,concept_period_type,unit_unit,decimals,concept_xml_id,concept_schema_url
88,CommonStockSharesAuthorized,2020-12-31,10000000.0,NaN,instant,shares,NaN,us-gaap_CommonStockSharesAuthorized,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
89,CommonStockSharesAuthorized,2021-06-30,10000000.0,NaN,instant,shares,NaN,us-gaap_CommonStockSharesAuthorized,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
90,CommonStockSharesIssued,2020-12-31,10000000.0,NaN,instant,shares,NaN,us-gaap_CommonStockSharesIssued,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
91,CommonStockSharesIssued,2021-06-30,10000000.0,NaN,instant,shares,NaN,us-gaap_CommonStockSharesIssued,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
92,CommonStockSharesOutstanding,2020-12-31,10000000.0,NaN,instant,shares,NaN,us-gaap_CommonStockSharesOutstanding,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
93,CommonStockSharesOutstanding,2021-06-30,10000000.0,NaN,instant,shares,NaN,us-gaap_CommonStockSharesOutstanding,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
25,EntityCommonStockSharesOutstanding,2021-08-09,10000000.0,NaN,instant,shares,NaN,dei_EntityCommonStockSharesOutstanding,https://xbrl.sec.gov/dei/2020/dei-2020-01-31.xsd


# Ratio Data 
* Filers also disclose ratios.
* Ratios in this example filing are both fiscal instant and duration disclosures. 

In [53]:
df_cols = ['concept_name','context_start_date','context_end_date','context_instant_date','value','concept_balance','concept_period_type','unit_unit','decimals', 
          'concept_xml_id', 'concept_schema_url']

df = GAAP_csv[df_cols][(GAAP_csv.unit_unit == 'pure')]

display_full_dataframe(df)

,concept_name,context_start_date,context_end_date,context_instant_date,value,concept_balance,concept_period_type,unit_unit,decimals,concept_xml_id,concept_schema_url
324,AlternativeInvestmentsAsPercentageOfOverallInv...,2021-01-01,2021-06-30,NaT,0.02,NaN,duration,pure,2.0,cik0000726865_AlternativeInvestmentsAsPercenta...,https://www.sec.gov/Archives/edgar/data/726865...
325,AlternativeInvestmentsAsPercentageOfOverallInv...,2020-01-01,2020-12-31,NaT,0.01,NaN,duration,pure,2.0,cik0000726865_AlternativeInvestmentsAsPercenta...,https://www.sec.gov/Archives/edgar/data/726865...
344,CollateralObtainedPercentOfFairValueSecurities...,NaT,NaT,2021-06-30,1.02,NaN,instant,pure,2.0,cik0000726865_CollateralObtainedPercentOfFairV...,https://www.sec.gov/Archives/edgar/data/726865...
345,CollateralObtainedPercentOfFairValueSecurities...,NaT,NaT,2021-06-30,1.05,NaN,instant,pure,2.0,cik0000726865_CollateralObtainedPercentOfFairV...,https://www.sec.gov/Archives/edgar/data/726865...
446,EffectiveIncomeTaxRateContinuingOperations,2021-04-01,2021-06-30,NaT,0.15,NaN,duration,pure,2.0,us-gaap_EffectiveIncomeTaxRateContinuingOperat...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
447,EffectiveIncomeTaxRateContinuingOperations,2021-01-01,2021-06-30,NaT,0.16,NaN,duration,pure,2.0,us-gaap_EffectiveIncomeTaxRateContinuingOperat...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
448,EffectiveIncomeTaxRateContinuingOperations,2020-04-01,2020-06-30,NaT,0.36,NaN,duration,pure,2.0,us-gaap_EffectiveIncomeTaxRateContinuingOperat...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
449,EffectiveIncomeTaxRateContinuingOperations,2020-01-01,2020-06-30,NaT,0.05,NaN,duration,pure,2.0,us-gaap_EffectiveIncomeTaxRateContinuingOperat...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
450,EffectiveIncomeTaxRateReconciliationAtFederalS...,2020-01-01,2020-06-30,NaT,0.21,NaN,duration,pure,NaN,us-gaap_EffectiveIncomeTaxRateReconciliationAt...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...
451,EffectiveIncomeTaxRateReconciliationAtFederalS...,2020-04-01,2020-06-30,NaT,0.21,NaN,duration,pure,NaN,us-gaap_EffectiveIncomeTaxRateReconciliationAt...,http://xbrl.fasb.org/us-gaap/2020/elts/us-gaap...


# GAAP Data Collection and Coverages
* The current filing and GAAP data collection includes over 81,000 filings from 2019 - Present.
* Filings are captured in near real time using the SEC RSS feed and Edgar data repositories. 

In [67]:
import os, os.path
import Source.MonitorFilings.Parameters as params

def get_percent(ct, tot):
    pct = (ct / tot) * 100
    return round(pct, 2)

total_filings = len(os.listdir(params.sec_raw_filings_dir))
print('Total 10Q Filings:' , total_filings)

Total 10Q Filings: 81953


### Get all the GAAP File Paths for the Example Filing Above

In [61]:
from Source.GAAP.GaapFeatures import * 

# Get the file paths for all of the GAAP feature files we just created
GAAP_paths = get_GAAP_file_paths(file_name)

GAAP_paths.keys()

dict_keys(['path_10Q_XBRL_Instance', 'path_10Q_GAAP_Raw', 'path_10Q_GAAP_Features'])

# XBRL instance files 
The XBRL instance document contains all of the namespaces, taxonomies, filing data, and metadata provided by each company for a given filing. 

In [70]:
xbrl_instances = len(os.listdir(params.XBRL_instance_dir))
print('Total 10Q Xbrl Instances:' , xbrl_instances)
print('Xbrl Instance Coverage:', get_percent(xbrl_instances, total_filings))

Total 10Q Xbrl Instances: 81885
Xbrl Instance Coverage: 99.92


**A sample of the example filing's extracted XBRL instance document:**

In [71]:
with open(GAAP_paths['path_10Q_XBRL_Instance'], 'r') as file:
    xbrl_instance = file.read()
    
print(xbrl_instance[:10000])

<?xml version="1.0" encoding="utf-8"?>
<xbrl
  xmlns="http://www.xbrl.org/2003/instance"
  xmlns:cik0000726865="http://lfg.com/20210630"
  xmlns:dei="http://xbrl.sec.gov/dei/2020-01-31"
  xmlns:iso4217="http://www.xbrl.org/2003/iso4217"
  xmlns:link="http://www.xbrl.org/2003/linkbase"
  xmlns:srt="http://fasb.org/srt/2020-01-31"
  xmlns:stpr="http://xbrl.sec.gov/stpr/2018-01-31"
  xmlns:us-gaap="http://fasb.org/us-gaap/2020-01-31"
  xmlns:xbrldi="http://xbrl.org/2006/xbrldi"
  xmlns:xlink="http://www.w3.org/1999/xlink"
  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
    <link:schemaRef xlink:href="c865-20210630.xsd" xlink:type="simple"/>
    <context id="Duration_1_1_2021_To_6_30_2021_srt_OwnershipAxis_cik0000726865_LincolnLifeAnnuityCompanyOfNewYorkMember">
        <entity>
            <identifier scheme="http://www.sec.gov/CIK">0000726865</identifier>
            <segment>
                <xbrldi:explicitMember dimension="srt:OwnershipAxis">cik0000726865:LincolnLifeAnnuityCo

# Raw GAAP Fact / Feature Files
* One .csv file per filing is created containing all financial and metadata extracted from the XBRL instance document.  
* Each row contains the data feature name, value, debit / credit, ratio, decimal place precision, all available labels and descriptions, denominations (USD, shares etc), taxonomy and namespace urls, and context instant, start, and end dates.  
* A row in this file could be filing metadata like a CIK number, company name, or ticker.  
* It could also be a fiscal duration or instant data element from the balance sheet.  
* You could literally recreate all digital tables in the filing and / or the company balance sheet using this file.

In [72]:
gaap_fact_files = len(os.listdir(params.GAAP_dir))
print('Total 10Q GAAP Fact Files:' , gaap_fact_files)
print('GAAP Fact File Coverage:', get_percent(gaap_fact_files, total_filings))

Total 10Q GAAP Fact Files: 79816
GAAP Fact File Coverage: 97.39


### GAAP Facts Sample- Example Filing

In [76]:
import pandas as pd

date_cols = ['context_start_date','context_end_date','context_instant_date']
GAAP_raw = pd.read_csv(GAAP_paths['path_10Q_GAAP_Raw'], parse_dates=date_cols)

with pd.option_context('display.max_columns', None):
    display(GAAP_raw.head(5))

,concept_xml_id,concept_schema_url,concept_name,concept_substitution_group,concept_concept_type,concept_abstract,concept_nillable,concept_period_type,concept_balance,concept_labels_0_label,concept_labels_0_language,concept_labels_0_text,concept_labels_0_label_type,concept_type,context_xml_id,context_entity,context_start_date,context_end_date,value,xml_id,footnote,context_instant_date,unit_unit_id,unit_unit,decimals,concept_labels_1_label,concept_labels_1_language,concept_labels_1_text,concept_labels_1_label_type,concept_labels_2_label,concept_labels_2_language,concept_labels_2_text,concept_labels_2_label_type,concept_labels_3_label,concept_labels_3_language,concept_labels_3_text,concept_labels_3_label_type,concept_labels_4_label,concept_labels_4_language,concept_labels_4_text,concept_labels_4_label_type
0,dei_AmendmentFlag,https://xbrl.sec.gov/dei/2020/dei-2020-01-31.xsd,AmendmentFlag,item,NaN,False,True,duration,NaN,dei_AmendmentFlag_lbl,en-US,Amendment Flag,http://www.xbrl.org/2003/role/label,xbrli:booleanItemType,Duration_1_1_2021_To_6_30_2021,726865,2021-01-01,2021-06-30,false,ct-nonNumeric-88688092-ee75-41c1-a4b5-bc0333c6...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,dei_CurrentFiscalYearEndDate,https://xbrl.sec.gov/dei/2020/dei-2020-01-31.xsd,CurrentFiscalYearEndDate,item,NaN,False,True,duration,NaN,dei_CurrentFiscalYearEndDate_lbl,en-US,Current Fiscal Year End Date,http://www.xbrl.org/2003/role/label,xbrli:gMonthDayItemType,Duration_1_1_2021_To_6_30_2021,726865,2021-01-01,2021-06-30,--12-31,ct-nonNumeric-5eec7d87-668b-4a48-b1d0-ea69d076...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,dei_DocumentFiscalPeriodFocus,https://xbrl.sec.gov/dei/2020/dei-2020-01-31.xsd,DocumentFiscalPeriodFocus,item,NaN,False,True,duration,NaN,dei_DocumentFiscalPeriodFocus_lbl,en-US,Document Fiscal Period Focus,http://www.xbrl.org/2003/role/label,dei:fiscalPeriodItemType,Duration_1_1_2021_To_6_30_2021,726865,2021-01-01,2021-06-30,Q2,ct-nonNumeric-c8f04bb5-415d-4c56-8797-aca2d00e...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,dei_EntityCentralIndexKey,https://xbrl.sec.gov/dei/2020/dei-2020-01-31.xsd,EntityCentralIndexKey,item,NaN,False,True,duration,NaN,dei_EntityCentralIndexKey_lbl,en-US,Entity Central Index Key,http://www.xbrl.org/2003/role/label,dei:centralIndexKeyItemType,Duration_1_1_2021_To_6_30_2021,726865,2021-01-01,2021-06-30,0000726865,ct-nonNumeric-7a1b7e9d-60f8-44c0-90f6-8fc747ec...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dei_EntityCurrentReportingStatus,https://xbrl.sec.gov/dei/2020/dei-2020-01-31.xsd,EntityCurrentReportingStatus,item,NaN,False,True,duration,NaN,dei_EntityCurrentReportingStatus_lbl,en-US,Entity Current Reporting Status,http://www.xbrl.org/2003/role/label,dei:yesNoItemType,Duration_1_1_2021_To_6_30_2021,726865,2021-01-01,2021-06-30,Yes,ct-nonNumeric-769e388f-04f5-47a5-9827-9a27801f...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Gaap Features Library
The GaapFeatures.py library extracts all XBRL facts for each filing.  Facts are then organized into multiple GAAP feature views stored at the filing level and in the full 10Q_GAAP_Features data repo. 

### 10Q_GAAP_Features Data Repo
* Dictionary file containing all current views of the GAAP and filing data.  
* The coverage number below indicates that at least one view exists per filing.
* Some filings are non-compliant and do not contain a XBRL instance document. 
* Other filings may disclose only metadata or very limited financial data. 
* As a result, some filing's 10Q_GAAP_Features file may not contain all possible views. 

In [77]:
gaap_feature_files = len(os.listdir(params.GAAP_features_dir))
print('Total 10Q GAAP Feature Files:' , gaap_feature_files)
print('GAAP Fact File Coverage:', get_percent(gaap_feature_files, total_filings))

Total 10Q GAAP Feature Files: 79733
GAAP Fact File Coverage: 97.29


### GAAP Feature Views Currently Processed
The following list contains all of the GAAP feature views that are currently supported.  The remaining sections show coverages, definitions, and examples for each view type.

In [80]:
from Source.MonitorFilings.PersistedParameters import load_from_pickle

GAAP_features = load_from_pickle(GAAP_paths['path_10Q_GAAP_Features'])

print('\nCurrent GAAP Feature Views Available:\n')

for k in GAAP_features.keys():
    print(k)


Current GAAP Feature Views Available:

gaap_metadata
gaap_FD
gaap_FI
QoQ
YoY
FYED_to_DPED
QoQ_pct
YoY_pct
FYED_to_DPED_pct
gaap_pct_features


### GAAP Feature Library Counts
Here I inspect the GAAP feature file for every single filing and count the GAAP feature view coverages.

In [99]:
# Create and init all count keys to 0 
gaap_feature_counts = {}
gaap_feature_counts['gaap_feature_files'] = 0
gaap_feature_counts['gaap_metadata'] = 0
gaap_feature_counts['gaap_FD'] = 0
gaap_feature_counts['gaap_FD_GE_2'] = 0
gaap_feature_counts['gaap_FI'] = 0
gaap_feature_counts['gaap_FI_GE_2'] = 0 
gaap_feature_counts['QoQ'] = 0 
gaap_feature_counts['YoY'] = 0
gaap_feature_counts['FYED_to_DPED'] = 0
gaap_feature_counts['QoQ_pct'] = 0
gaap_feature_counts['YoY_pct'] = 0
gaap_feature_counts['FYED_to_DPED_pct'] = 0 
gaap_feature_counts['gaap_pct_features'] = 0 

# Count coverages for all GAAP_Feature files
for f_name in os.listdir(params.GAAP_features_dir):
    
    gf = load_from_pickle(params.GAAP_features_dir + f_name)
    
    # Count each file processed
    gaap_feature_counts['gaap_feature_files'] += 1 
    
    # Count metadata files
    if gf.get('gaap_metadata') is not None:
        gaap_feature_counts['gaap_metadata'] += 1 
        
    # Count fiscal duration data records
    gaap_FD = gf.get('gaap_FD')
    
    if gaap_FD is not None:
        gaap_feature_counts['gaap_FD'] += 1 
        
        if len(gaap_FD) >= 2:
            gaap_feature_counts['gaap_FD_GE_2'] += 1
            
            # No need to count FD matches, unless there are at least 2 FD records
            if gf.get('QoQ') is not None:
                gaap_feature_counts['QoQ'] += 1 
                
            if gf.get('YoY') is not None:
                gaap_feature_counts['YoY'] += 1 
                
            if gf.get('QoQ_pct') is not None:
                gaap_feature_counts['QoQ_pct'] += 1 
                
            if gf.get('YoY_pct') is not None:
                gaap_feature_counts['YoY_pct'] += 1 
            
    # Count fiscal instancce data records
    gaap_FI = gf.get('gaap_FI')
    
    if gaap_FI is not None:
        gaap_feature_counts['gaap_FI'] += 1 
        
        if len(gaap_FI) >= 2:
            gaap_feature_counts['gaap_FI_GE_2'] += 1
            
            # No need to count FI matches, unless there are at least 2 FI records
            if gf.get('FYED_to_DPED') is not None:
                gaap_feature_counts['FYED_to_DPED'] += 1
            
            if gf.get('FYED_to_DPED_pct') is not None:
                gaap_feature_counts['FYED_to_DPED_pct'] += 1
                
    if gf.get('gaap_pct_features') is not None:
        gaap_feature_counts['gaap_pct_features'] += 1

gaap_feature_counts

{'gaap_feature_files': 79733,
 'gaap_metadata': 79733,
 'gaap_FD': 79733,
 'gaap_FD_GE_2': 79341,
 'gaap_FI': 79733,
 'gaap_FI_GE_2': 79522,
 'QoQ': 76367,
 'YoY': 75655,
 'FYED_to_DPED': 78605,
 'QoQ_pct': 76367,
 'YoY_pct': 75655,
 'FYED_to_DPED_pct': 78605,
 'gaap_pct_features': 79733}

# GAAP Feature Coverages
Based on the counts above, we have:

**>= 97% coverage for:**
 * GAAP feature files
 * Filing metadata
 * Fiscal Durations 
 * Fiscal Instances
 
**>= 95% Coverage for:**
 * Fiscal year end instance data
 * Document period end instance data 
 * FYED_to_DPED instance matches - Compares the balance sheet data (which provides a snapshot of the company's financial position at a specific point in time) at the end of the fiscal year with the balance sheet data at the end of the reporting period for the 10-Q filing.
 * FYED_to_DPED_pct - Percentage data for all instance data elements that occur in both the fiscal year end and document period ending instance data. 
 
**>= 92% Coverage for:** 
 * Year over year and YoY percentage duration data
 * YTD over YTD and YTD percentage duration data

In [104]:
for k, v in gaap_feature_counts.items():
    print(k, '\nCount:',v,'Coverage:', get_percent(v, total_filings),'\n')

gaap_feature_files 
Count: 79733 Coverage: 97.29 

gaap_metadata 
Count: 79733 Coverage: 97.29 

gaap_FD 
Count: 79733 Coverage: 97.29 

gaap_FD_GE_2 
Count: 79341 Coverage: 96.81 

gaap_FI 
Count: 79733 Coverage: 97.29 

gaap_FI_GE_2 
Count: 79522 Coverage: 97.03 

QoQ 
Count: 76367 Coverage: 93.18 

YoY 
Count: 75655 Coverage: 92.32 

FYED_to_DPED 
Count: 78605 Coverage: 95.91 

QoQ_pct 
Count: 76367 Coverage: 93.18 

YoY_pct 
Count: 75655 Coverage: 92.32 

FYED_to_DPED_pct 
Count: 78605 Coverage: 95.91 

gaap_pct_features 
Count: 79733 Coverage: 97.29 



# Filing Metadata View
In addition to GAAP data, filers disclose a substancial amount of other filing metadata as well. 

**A few examples include:** 
 * Company Name
 * Ticker
 * CurrentFiscalYearEndDate
 * DocumentPeriodEndDate
 * Flags (AmendmentFlag, EntitySmallBusiness, EntityEmergingGrowthCompany) etc.
 * Periods (LesseeFinanceLeaseRenewalTerm1, FinanceLeaseWeightedAverageRemainingLeaseTerm1)
 * Categories (i.e. EntityFilerCategory)

### Metadata View - Example Filing
**Filing metadata is currently captured for all fillings with 97.29% coverage.**

In [81]:
GAAP_features['gaap_metadata']

{'AmendmentFlag': 'false',
 'CurrentFiscalYearEndDate': '--12-31',
 'DocumentFiscalPeriodFocus': 'Q2',
 'EntityCentralIndexKey': '0000726865',
 'EntityCurrentReportingStatus': 'Yes',
 'EntityInteractiveDataCurrent': 'Yes',
 'DocumentType': '10-Q',
 'DocumentQuarterlyReport': 'true',
 'DocumentPeriodEndDate': '2021-06-30',
 'DocumentFiscalYearFocus': '2021',
 'DocumentTransitionReport': 'false',
 'EntityFileNumber': '000-55871',
 'EntityRegistrantName': 'THE LINCOLN NATIONAL LIFE INSURANCE COMPANY',
 'EntityIncorporationStateCountryCode': 'IN',
 'EntityTaxIdentificationNumber': '35-0472300',
 'EntityAddressAddressLine1': '1300 South Clinton Street',
 'EntityAddressCityOrTown': 'Fort Wayne',
 'EntityAddressStateOrProvince': 'IN',
 'EntityAddressPostalZipCode': '46802',
 'CityAreaCode': '260',
 'LocalPhoneNumber': '455-2000',
 'EntityFilerCategory': 'Non-accelerated Filer',
 'EntitySmallBusiness': 'false',
 'EntityEmergingGrowthCompany': 'false',
 'EntityShellCompany': 'false'}

# Fiscal Durations View 
* The fiscal durations view is a pandas dataframe containing all fiscal durations disclosed within a filing.
* Each row represents a separate fiscal duration and includes the durations context start and end date. 
* Dataframe columns contain all fiscal duration features disclosed in a filing.
* Fiscal duration features that are missing from a particular duration will contain a value of NaN. 
* Rows are sorted by the context start and end dates.

**Scrolling to the far right the fiscal duration view also shows:**
 * **duration_days** - The number days in the duration 
 * **duration_YTD_days_dist** - The absolute distance in days from the filings YTD days of (90, 180, 270, 360) based on quarter.
 * **duration_fiscal_YTD_days_dist** - The absolute distance in days from the filings fiscal YTD days based on the last fiscal year end date and the document period end date.
 * **duration_90_days_dist** - The absolute distance in days from 90 days. 

### Fiscal Durations View - Example Filing
**Filings reporting at least 1 fiscal duration:**
 * Count: 79733 
 * Coverage: 97.29%
 
**Filings reporting at >= 2 fiscal durations:** 
 * Count: 79341 
 * Coverage: 96.81%

In [106]:
GAAP_features['gaap_FD']

concept_name,context_start_date,context_end_date,AffordableHousingTaxCreditsAndOtherTaxBenefitsAmount,AffordableHousingTaxCreditsAndOtherTaxExpenseAmount,AlternativeInvestmentsAsPercentageOfOverallInvestedAssets,AmortizationAssociatedWithRealizedGainLossOnCertainInvestments,AmortizationOfDeferredGainOnBusinessSoldThroughReinsurance,AvailableForSaleSecuritiesChangeInGrossUnrealizedHoldingGainLoss,BenefitsLossesAndExpenses,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect,...,SupplementalInformationForPropertyCasualtyInsuranceUnderwritersCurrentYearClaimsAndClaimsAdjustmentExpense,TotalRealizedGainLossRelatedToCertainInvestments,TransfersToAndFromSeparateAccountsNet,VariableAnnuityNetDerivativesResultsAssociatedAmortizationIncomeExpense,VariableAnnuityNetDerivativesResultsGrossGainLoss,WithdrawalsOfFixedAccountValuesIncludingFixedPortionOfVariable,duration_days,duration_YTD_days_dist,duration_fiscal_YTD_days_dist,duration_90_days_dist
0,2020-01-01,2020-06-30,NaN,-1000000.0,NaN,43000000.0,18000000.0,NaN,7.397000e+09,1.340000e+09,...,1.742000e+09,-222000000.0,-216000000.0,17000000.0,-110000000.0,3.291000e+09,181 days,1 days,0 days,91 days
1,2020-01-01,2020-12-31,NaN,NaN,0.01,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,365 days,185 days,184 days,275 days
2,2020-04-01,2020-06-30,NaN,NaN,NaN,18000000.0,9000000.0,NaN,3.757000e+09,NaN,...,NaN,-138000000.0,NaN,8000000.0,-51000000.0,NaN,90 days,90 days,91 days,0 days
3,2021-01-01,2021-06-30,2000000.0,NaN,0.02,-10000000.0,14000000.0,120000000.0,8.018000e+09,4.520000e+08,...,1.932000e+09,51000000.0,229000000.0,16000000.0,-113000000.0,3.283000e+09,180 days,0 days,1 days,90 days
4,2021-04-01,2021-06-30,1000000.0,NaN,NaN,-5000000.0,7000000.0,NaN,3.891000e+09,NaN,...,NaN,19000000.0,NaN,8000000.0,-62000000.0,NaN,90 days,90 days,91 days,0 days


### Durations Disclosed in the Example Filing
When considering the filing metadata, context_start_date, context_end_date, and duration_days we see the following fiscal durations disclosed in the example filing:

**Last Fiscal Year Duration**
* CurrentFiscalYearEndDate: --12-31
* DocumentPeriodEndDate: 2021-06-30
* Last fiscal year end: 2020-12-31 
* Last Fiscal Year Duration: 2020-01-01 to 2020-12-31 	

**Year-over-Year (QoQ) Quarterly Fiscal Durations**
 * The most recent quarterly duration disclosed is: 2021-04-01 to 2021-06-30 (90 days)	
 * The prior year quarterly duration disclosed is: 2020-04-01 to 2020-06-30 (90 days)
 * The dates and / or days do not always match in other filings.  
 
**YTD-over-YTD (YoY) Year to Date Fiscal Durations**
 * The DocumentFiscalPeriodFocus is Q2 
 * The fiscal YTD days is 180
 * The most recent YTD duration disclosed is: 2021-01-01 to 2021-06-30 (180 days)
 * The prior year YTD duration disclosed is: 2020-01-01	to 2020-06-30 (181 days)		
 * The dates and / or days do not always match in other filings.  


In [110]:
metadata = GAAP_features['gaap_metadata'] 
print('CurrentFiscalYearEndDate:', metadata['CurrentFiscalYearEndDate'])
print('DocumentPeriodEndDate:', metadata['DocumentPeriodEndDate'])
print('DocumentFiscalPeriodFocus:', metadata['DocumentFiscalPeriodFocus'])

GAAP_features['gaap_FD'][['context_start_date','context_end_date','duration_days']]

CurrentFiscalYearEndDate: --12-31
DocumentPeriodEndDate: 2021-06-30
DocumentFiscalPeriodFocus: Q2


concept_name,context_start_date,context_end_date,duration_days
0,2020-01-01,2020-06-30,181 days
1,2020-01-01,2020-12-31,365 days
2,2020-04-01,2020-06-30,90 days
3,2021-01-01,2021-06-30,180 days
4,2021-04-01,2021-06-30,90 days


# Matched Fiscal Duration Views

**The GaapFeatures library currently matches two different types of fiscal durations for all filings:**
* QoQ - The QoQ view currently captures the most recent quarterly duration disclosed on a filing and the same period for the prior year when they exist. 
* YoY - The YoY view currently captures the most recent YTD duration disclosed on a filing and the same period for the prior year when they exist. 

**Year over Year Quarterly Durations QoQ** 
* Count: 76367 
* Coverage: 93.18%

**Fiscal YTD over YTD Durations YoY**
* Count: 75655 
* Coverage: 92.32%

**Matching these durations is complex for a number of reasons including:**
* Start and end dates disclosed do not always match the expected quarterly or YTD ranges when considering the fiscal year end and document period end. In many cases, the dates will be within 7 days of the expected date.
* Quarterly durations are not always 90 days.
* Year over year quarterly durations do not always have the same number of days. For example, the most recent quarter may cover 90 days and the prior year's quarter may cover 94 days. 
* Fiscal YTD durations change based on the current fiscal quarter.
* Fiscal YTD durations do not always cover exactly (90, 180, 270, or 360) days.
* Matched fiscal YTD durations do not necessarily cover the same exact number of days. 

In [127]:
GAAP_features['QoQ']

concept_name,context_start_date,context_end_date,AmortizationAssociatedWithRealizedGainLossOnCertainInvestments,AmortizationOfDeferredGainOnBusinessSoldThroughReinsurance,BenefitsLossesAndExpenses,ComprehensiveIncomeNetOfTax,DebtAndEquitySecuritiesRealizedGainLoss,DebtSecuritiesAvailableForSaleAllowanceForCreditLossWriteoff,EffectiveIncomeTaxRateContinuingOperations,EffectiveIncomeTaxRateReconciliationAtFederalStatutoryIncomeTaxRate,...,PercentOfPermanentLifeInsuranceSales,PolicyholderBenefitsAndClaimsIncurredNet,PremiumsEarnedNet,ProvisionForLoanLossesExpensedAllowanceRecorded,RealizedGainLossOnMarkToMarketOnCertainInstruments,Revenues,StrategicDigitizationExpense,TotalRealizedGainLossRelatedToCertainInvestments,VariableAnnuityNetDerivativesResultsAssociatedAmortizationIncomeExpense,VariableAnnuityNetDerivativesResultsGrossGainLoss
2,2020-04-01,2020-06-30,18000000.0,9000000.0,3.757000e+09,3.897000e+09,-433000000.0,15000000.0,0.36,0.21,...,0.37,1.870000e+09,1.285000e+09,111000000.0,-235000000.0,3.531000e+09,14000000.0,-138000000.0,8000000.0,-51000000.0
4,2021-04-01,2021-06-30,-5000000.0,7000000.0,3.891000e+09,2.183000e+09,-172000000.0,6000000.0,0.15,0.21,...,0.15,1.848000e+09,1.328000e+09,-14000000.0,-131000000.0,4.435000e+09,21000000.0,19000000.0,8000000.0,-62000000.0


In [133]:
print('Financial Features Captured for the QoQ view:',len(GAAP_features['QoQ'].columns))

Financial Features Captured for the QoQ view: 47


In [134]:
GAAP_features['YoY']

concept_name,context_start_date,context_end_date,AmortizationAssociatedWithRealizedGainLossOnCertainInvestments,AmortizationOfDeferredGainOnBusinessSoldThroughReinsurance,BenefitsLossesAndExpenses,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect,CashFlowHedgeReclassifiedToEarningsNet,CommonStockIssuedForBenefitPlansAndExcessTaxBenefits,ComprehensiveIncomeNetOfTax,DebtAndEquitySecuritiesRealizedGainLoss,...,RepaymentAndMaturitiesOfMortgageLoansOnRealEstate,RepaymentsOfLongTermDebt,Revenues,StrategicDigitizationExpense,SupplementalInformationForPropertyCasualtyInsuranceUnderwritersCurrentYearClaimsAndClaimsAdjustmentExpense,TotalRealizedGainLossRelatedToCertainInvestments,TransfersToAndFromSeparateAccountsNet,VariableAnnuityNetDerivativesResultsAssociatedAmortizationIncomeExpense,VariableAnnuityNetDerivativesResultsGrossGainLoss,WithdrawalsOfFixedAccountValuesIncludingFixedPortionOfVariable
0,2020-01-01,2020-06-30,43000000.0,18000000.0,7.397000e+09,1.340000e+09,0.0,7000000.0,2.337000e+09,-435000000.0,...,596000000.0,30000000.0,7.772000e+09,26000000.0,1.742000e+09,-222000000.0,-216000000.0,17000000.0,-110000000.0,3.291000e+09
3,2021-01-01,2021-06-30,-10000000.0,14000000.0,8.018000e+09,4.520000e+08,0.0,11000000.0,-5.340000e+08,39000000.0,...,846000000.0,60000000.0,9.124000e+09,35000000.0,1.932000e+09,51000000.0,229000000.0,16000000.0,-113000000.0,3.283000e+09


In [135]:
print('Financial Features Captured for the YTD view:',len(GAAP_features['YoY'].columns))

Financial Features Captured for the YTD view: 97


# Percentage Change Duration Views 
**Percentage changes are calculated for all financial features disclosed in both views.**

**Year over Year Quarterly Percentages QoQ_pct** 
* Count: 76367 
* Coverage: 93.18%  

**Fiscal YTD over YTD Percentages YoY_pct**
* Count: 75655 
* Coverage: 92.32% 

In [136]:
GAAP_features['QoQ_pct']

,period_start_QoQ,period_end_QoQ,AmortizationAssociatedWithRealizedGainLossOnCertainInvestments_QoQ,AmortizationOfDeferredGainOnBusinessSoldThroughReinsurance_QoQ,BenefitsLossesAndExpenses_QoQ,ComprehensiveIncomeNetOfTax_QoQ,DebtAndEquitySecuritiesRealizedGainLoss_QoQ,DebtSecuritiesAvailableForSaleAllowanceForCreditLossWriteoff_QoQ,EffectiveIncomeTaxRateContinuingOperations_QoQ,EffectiveIncomeTaxRateReconciliationAtFederalStatutoryIncomeTaxRate_QoQ,...,PercentOfPermanentLifeInsuranceSales_QoQ,PolicyholderBenefitsAndClaimsIncurredNet_QoQ,PremiumsEarnedNet_QoQ,ProvisionForLoanLossesExpensedAllowanceRecorded_QoQ,RealizedGainLossOnMarkToMarketOnCertainInstruments_QoQ,Revenues_QoQ,StrategicDigitizationExpense_QoQ,TotalRealizedGainLossRelatedToCertainInvestments_QoQ,VariableAnnuityNetDerivativesResultsAssociatedAmortizationIncomeExpense_QoQ,VariableAnnuityNetDerivativesResultsGrossGainLoss_QoQ
0,2020-04-01,2021-06-30,-1.277778,-0.222222,0.035667,-0.439826,-0.602771,-0.6,-0.583333,0.0,...,-0.594595,-0.011765,0.033463,-1.126126,-0.442553,0.256018,0.5,-1.137681,0.0,0.215686


In [140]:
print('Financial Features Captured for the QoQ percentage view:',len(GAAP_features['QoQ_pct'].columns))

Financial Features Captured for the QoQ percentage view: 47


In [141]:
GAAP_features['YoY_pct']

,period_start_YoY,period_end_YoY,AmortizationAssociatedWithRealizedGainLossOnCertainInvestments_YoY,AmortizationOfDeferredGainOnBusinessSoldThroughReinsurance_YoY,BenefitsLossesAndExpenses_YoY,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect_YoY,CashFlowHedgeReclassifiedToEarningsNet_YoY,CommonStockIssuedForBenefitPlansAndExcessTaxBenefits_YoY,ComprehensiveIncomeNetOfTax_YoY,DebtAndEquitySecuritiesRealizedGainLoss_YoY,...,RepaymentAndMaturitiesOfMortgageLoansOnRealEstate_YoY,RepaymentsOfLongTermDebt_YoY,Revenues_YoY,StrategicDigitizationExpense_YoY,SupplementalInformationForPropertyCasualtyInsuranceUnderwritersCurrentYearClaimsAndClaimsAdjustmentExpense_YoY,TotalRealizedGainLossRelatedToCertainInvestments_YoY,TransfersToAndFromSeparateAccountsNet_YoY,VariableAnnuityNetDerivativesResultsAssociatedAmortizationIncomeExpense_YoY,VariableAnnuityNetDerivativesResultsGrossGainLoss_YoY,WithdrawalsOfFixedAccountValuesIncludingFixedPortionOfVariable_YoY
0,2020-01-01,2021-06-30,-1.232558,-0.222222,0.083953,-0.662687,-1.0,0.571429,-1.228498,-1.089655,...,0.419463,1.0,0.173958,0.346154,0.10907,-1.22973,-2.060185,-0.058824,0.027273,-0.002431


In [142]:
print('Financial Features Captured for the YTD percentage view:',len(GAAP_features['YoY_pct'].columns))

Financial Features Captured for the YTD percentage view: 97


# Fiscal Instances View 
* The fiscal instance view is a pandas dataframe containing all fiscal instances disclosed within a filing.
* Each row represents a separate fiscal instant and includes the context_instant_date when the instance snapshot was taken. 
* Dataframe columns contain all fiscal instant features disclosed in a filing.
* Fiscal instant features that are missing from a particular context_instant_date will contain a value of NaN. 
* Rows are sorted by the context instant date.

**Scrolling to the far right the fiscal instance view also shows:**
 * **context_instant_days_diff** - The difference in days between each row's context instant date.
 * **period_end_date_days_diff** - The difference in days between context instant date and document period end date.
 * **days_from_dped** - The absolute difference in days between context instant date and document period end date.
 * **days_from_fyed** - The absolute difference in days between context instant date and the last fiscal year end date.

### Fiscal Instant View - Example Filing
**Fiscal Instances are currently captured for all fillings with 97.29% coverage.**

In [105]:
GAAP_features['gaap_FI']

concept_name,context_instant_date,AccruedInvestmentIncomeReceivable,AccumulatedOtherComprehensiveIncomeLossNetOfTax,Assets,AvailableForSaleDebtSecuritiesAmortizedCostBasis,AvailableForSaleSecuritiesDebtSecurities,AvailableforsaleSecuritiesInUnrealizedLossPositionsQualitativeDisclosureNumberOfPositions1,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents,CollateralObtainedPercentOfFairValueSecuritiesLendingAgreementsDomestic,CollateralObtainedPercentOfFairValueSecuritiesLendingAgreementsForeign,...,SecuritiesPledgedUnderSecuritiesLendingAgreementsFairValue,SeparateAccountAssets,SeparateAccountsLiability,StockholdersEquity,TotalPayablesForCollateralOnInvestmentFairValue,TradingSecuritiesDebt,context_instant_days_diff,period_end_date_days_diff,days_from_dped,days_from_fyed
0,2019-12-31,NaN,NaN,NaN,NaN,NaN,NaN,1.879000e+09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0 days,-547 days,547 days,366 days
1,2020-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,91 days,-456 days,456 days,275 days
2,2020-06-30,NaN,NaN,NaN,NaN,NaN,NaN,3.219000e+09,NaN,NaN,...,NaN,NaN,NaN,2.380200e+10,NaN,NaN,91 days,-365 days,365 days,184 days
3,2020-12-31,1.217000e+09,9.021000e+09,3.683190e+11,1.030210e+11,1.211110e+11,NaN,1.462000e+09,NaN,NaN,...,112000000.0,1.679650e+11,1.679650e+11,2.504100e+10,8.131000e+09,4.442000e+09,184 days,-181 days,181 days,0 days
4,2021-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,90 days,-91 days,91 days,90 days
5,2021-06-30,1.215000e+09,7.561000e+09,3.841280e+11,1.055300e+11,1.204520e+11,1353.0,1.914000e+09,1.02,1.05,...,222000000.0,1.787950e+11,1.787950e+11,2.408900e+10,1.013100e+10,4.175000e+09,91 days,0 days,0 days,181 days
6,2021-08-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,40 days,40 days,40 days,221 days


### Instances Disclosed in the Example Filing 
When considering the filing metadata, context_instant_date, days_from_dped, and days_from_fyed for the example filing we see the following fiscal instances disclosed in the example filing: 

**Quarter over Quarter snapshots are disclosed for the current quarter:**  
* DocumentPeriodEndDate: 2021-06-30
* QoQ 1: 2021-06-30 and 2020-06-30

**Quarter over Quarter snapshots are disclosed for the prior quarter:**  
* DocumentPeriodEndDate: 2021-06-30
* QoQ 2: 2021-03-31 and 2020-03-31 

**Year over year snapshots are disclosed for the Fiscal Year End Date:**
 * DocumentPeriodEndDate: 2021-06-30 
 * CurrentFiscalYearEndDate: --12-31 
 * Last Fiscal Year End: 2020-12-31
 * Fiscal YoY Instants: 2020-12-31 and 2019-12-31	
 
**Note:**

After closer inspection of the data it also appears that the filer disclosed EntityCommonStockSharesOutstanding as of 2021-08-09. 

In [114]:
metadata = GAAP_features['gaap_metadata'] 
print('CurrentFiscalYearEndDate:', metadata['CurrentFiscalYearEndDate'])
print('DocumentPeriodEndDate:', metadata['DocumentPeriodEndDate'])
print('DocumentFiscalPeriodFocus:', metadata['DocumentFiscalPeriodFocus'])

GAAP_features['gaap_FI'][['context_instant_date', 'context_instant_days_diff', 'days_from_dped', 'days_from_fyed']]

CurrentFiscalYearEndDate: --12-31
DocumentPeriodEndDate: 2021-06-30
DocumentFiscalPeriodFocus: Q2


concept_name,context_instant_date,context_instant_days_diff,days_from_dped,days_from_fyed
0,2019-12-31,0 days,547 days,366 days
1,2020-03-31,91 days,456 days,275 days
2,2020-06-30,91 days,365 days,184 days
3,2020-12-31,184 days,181 days,0 days
4,2021-03-31,90 days,91 days,90 days
5,2021-06-30,91 days,0 days,181 days
6,2021-08-09,40 days,40 days,221 days


In [126]:
print('\nReview of items disclosed on 2021-08-09:')

gfi = GAAP_features['gaap_FI']
gfi = gfi[gfi.context_instant_date == '2021-08-09'].T
gfi.columns = ['concept_value']

gfi[gfi.concept_value.isnull() == False]


Review of items disclosed on 2021-08-09:


,concept_value
concept_name,
context_instant_date,2021-08-09 00:00:00
EntityCommonStockSharesOutstanding,10000000.0
context_instant_days_diff,40 days 00:00:00
period_end_date_days_diff,40 days 00:00:00
days_from_dped,40 days 00:00:00
days_from_fyed,221 days 00:00:00


# Matched Fiscal Instance Views

**The GaapFeatures library currently matches one type of fiscal instances for all filings:**
* FYED_to_DPED - Matched fiscal instance data for the prior fiscal year end and the document period end.  

 **Fiscal Year-End Date (FYED) to Document Period End Date (DPED):** 
 * Count: 78605 
 * Coverage: 95.91% 

In [145]:
GAAP_features['FYED_to_DPED']

concept_name,context_instant_date,AccruedInvestmentIncomeReceivable,AccumulatedOtherComprehensiveIncomeLossNetOfTax,Assets,AvailableForSaleDebtSecuritiesAmortizedCostBasis,AvailableForSaleSecuritiesDebtSecurities,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents,CollateralPayableHeldForDerivativeInvestmentsCarryingValue,CollateralPayableHeldForDerivativeInvestmentsFairValue,CommonStockSharesAuthorized,...,SecuritiesLoanedGross,SecuritiesLoanedIncludingNotSubjectToMasterNettingArrangementAndAssetsOtherThanSecuritiesTransferred,SecuritiesPledgedForFederalHomeLoanBankOfIndianapolisCarryingValue,SecuritiesPledgedForFederalHomeLoanBankOfIndianapolisFairValue,SecuritiesPledgedUnderSecuritiesLendingAgreementsFairValue,SeparateAccountAssets,SeparateAccountsLiability,StockholdersEquity,TotalPayablesForCollateralOnInvestmentFairValue,TradingSecuritiesDebt
3,2020-12-31,1.217000e+09,9.021000e+09,3.683190e+11,1.030210e+11,1.211110e+11,1.462000e+09,2.970000e+09,2.970000e+09,10000000.0,...,4.665000e+09,115000000.0,3.130000e+09,5.049000e+09,112000000.0,1.679650e+11,1.679650e+11,2.504100e+10,8.131000e+09,4.442000e+09
5,2021-06-30,1.215000e+09,7.561000e+09,3.841280e+11,1.055300e+11,1.204520e+11,1.914000e+09,4.384000e+09,4.384000e+09,10000000.0,...,7.482000e+09,228000000.0,3.580000e+09,5.525000e+09,222000000.0,1.787950e+11,1.787950e+11,2.408900e+10,1.013100e+10,4.175000e+09


In [146]:
print('Financial Features Captured for the FYED_to_DPED view:',len(GAAP_features['FYED_to_DPED'].columns))

Financial Features Captured for the FYED_to_DPED view: 92


# Percentage Change Instance Views 
**Percentage changes are calculated for all financial features disclosed in the FYED_to_DPED view.**

**Fiscal Year-End Date to Document Period End Date Percentages FYED_to_DPED_pct** 
* Count: 78605 
* Coverage: 95.91% 

In [147]:
GAAP_features['FYED_to_DPED_pct']

,period_start_FYED_to_DPED,period_end_FYED_to_DPED,AccruedInvestmentIncomeReceivable_FYED_to_DPED,AccumulatedOtherComprehensiveIncomeLossNetOfTax_FYED_to_DPED,Assets_FYED_to_DPED,AvailableForSaleDebtSecuritiesAmortizedCostBasis_FYED_to_DPED,AvailableForSaleSecuritiesDebtSecurities_FYED_to_DPED,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents_FYED_to_DPED,CollateralPayableHeldForDerivativeInvestmentsCarryingValue_FYED_to_DPED,CollateralPayableHeldForDerivativeInvestmentsFairValue_FYED_to_DPED,...,SecuritiesLoanedGross_FYED_to_DPED,SecuritiesLoanedIncludingNotSubjectToMasterNettingArrangementAndAssetsOtherThanSecuritiesTransferred_FYED_to_DPED,SecuritiesPledgedForFederalHomeLoanBankOfIndianapolisCarryingValue_FYED_to_DPED,SecuritiesPledgedForFederalHomeLoanBankOfIndianapolisFairValue_FYED_to_DPED,SecuritiesPledgedUnderSecuritiesLendingAgreementsFairValue_FYED_to_DPED,SeparateAccountAssets_FYED_to_DPED,SeparateAccountsLiability_FYED_to_DPED,StockholdersEquity_FYED_to_DPED,TotalPayablesForCollateralOnInvestmentFairValue_FYED_to_DPED,TradingSecuritiesDebt_FYED_to_DPED
0,2020-12-31,2021-06-30,-0.001643,-0.161845,0.042922,0.024354,-0.005441,0.309166,0.476094,0.476094,...,0.603859,0.982609,0.14377,0.094276,0.982143,0.064478,0.064478,-0.038018,0.245972,-0.060108


In [148]:
print('Financial Features Captured for the FYED_to_DPED percentage view:',len(GAAP_features['FYED_to_DPED_pct'].columns))

Financial Features Captured for the FYED_to_DPED percentage view: 93


# GAAP Percentage Features View
The GAAP percentage features view consolidates all filing metadata, QoQ_pct, YoY_pct, and FYED_to_DPED_pct data into a single consolidated view.
* Count: 79733 
* Coverage: 97.29% 

In [150]:
GAAP_features['gaap_pct_features']

,FileName,AmendmentFlag,CurrentFiscalYearEndDate,DocumentFiscalPeriodFocus,EntityCentralIndexKey,EntityCurrentReportingStatus,EntityInteractiveDataCurrent,DocumentType,DocumentQuarterlyReport,DocumentPeriodEndDate,...,SecuritiesLoanedGross_FYED_to_DPED,SecuritiesLoanedIncludingNotSubjectToMasterNettingArrangementAndAssetsOtherThanSecuritiesTransferred_FYED_to_DPED,SecuritiesPledgedForFederalHomeLoanBankOfIndianapolisCarryingValue_FYED_to_DPED,SecuritiesPledgedForFederalHomeLoanBankOfIndianapolisFairValue_FYED_to_DPED,SecuritiesPledgedUnderSecuritiesLendingAgreementsFairValue_FYED_to_DPED,SeparateAccountAssets_FYED_to_DPED,SeparateAccountsLiability_FYED_to_DPED,StockholdersEquity_FYED_to_DPED,TotalPayablesForCollateralOnInvestmentFairValue_FYED_to_DPED,TradingSecuritiesDebt_FYED_to_DPED
0,726865_0000726865-21-000388,false,--12-31,Q2,0000726865,Yes,Yes,10-Q,true,2021-06-30,...,0.603859,0.982609,0.14377,0.094276,0.982143,0.064478,0.064478,-0.038018,0.245972,-0.060108


In [151]:
print('Financial Features Captured for the gaap_pct_features view:',len(GAAP_features['gaap_pct_features'].columns))

Financial Features Captured for the gaap_pct_features view: 263


# GAAP Full Data Repository
Data from each individual filing's gaap_pct_features view is also propagated into the GAAP Data Repository full datasets. The GAAP Data Repository comprises datasets containing one row per filing for all filings including:  

* GAAP_metadata_10Q_Full - A pandas dataframe with one row per filing containing all filing metadata. 
* GAAP_Pct_Chg_10Q_Full - A scipy sparse csr matrix containing one row per filing with sparse feature columns for all unique percentage change features across all filings (currently 220,992 financial features). 
* GAAP_Pct_Chg_10Q_Full_Feature_Names - An index of all feature names represented in GAAP_Pct_Chg_10Q_Full.

These files will eventually be incorporated into the daily models used predicting stock price changes. 

### Current GAAP_Pct_Chg_10Q_Full Features for Machine Learning 

### The current GAAP_Pct_Chg_10Q_Full file contains: 
* 79,733 filings 
* 220,992 financial features  

In [155]:
from Source.MonitorFilings.PersistedParameters import load_from_pickle

GAAP_Pct_Chg_10Q_Full = load_from_pickle(params.GAAP_pct_chg_repo_path)

GAAP_Pct_Chg_10Q_Full

<79733x220992 sparse array of type '<class 'numpy.float64'>'
	with 11695268 stored elements in Compressed Sparse Row format>

### The current GAAP_Pct_Chg_10Q_Full_Feature_Names file contains:
* 220,992 financial features names   

In [157]:
GAAP_Pct_Chg_10Q_Full_Feature_Names = load_from_pickle(params.GAAP_pct_chg_feature_names_path)

print('Feature Names:',len(GAAP_Pct_Chg_10Q_Full_Feature_Names))

Feature Names: 220992


In [166]:
print('Feature Names Sample\n')
list(GAAP_Pct_Chg_10Q_Full_Feature_Names.keys())[:50]

Feature Names Sample



['a2020labordaywildfireearningstestreserve_qoq',
 'a2023notesequityissuancecosts_qoq',
 'abandonedprojectcosts_qoq',
 'abandonedprojectcostsrecovered_qoq',
 'abandonedprojectsexpense_qoq',
 'abandonedwellcosts_qoq',
 'abandonmentexpenseofoilandgasproperties_qoq',
 'abandonmentpaymentstoacquireintangibleassets_qoq',
 'abatementofrent_qoq',
 'abnormalcostscovid19productionsuspension_qoq',
 'aboveandbelowmarketleasesamortizationexpenseincome_qoq',
 'aboveandbelowmarketleasesamortizationincome_qoq',
 'abovemarketcomponentofswapextensions_qoq',
 'abovemarketleaseassetamortization_qoq',
 'abovemarketleaseperiodincreasedecrease_qoq',
 'abovemarketleasesamortizationexpense_qoq',
 'academicrevenue_qoq',
 'acceleratedallocatedsharebasedcompensationexpense_qoq',
 'acceleratedamortizationofdeferredcharges_qoq',
 'acceleratedamortizationofintangibleassets_qoq',
 'acceleratedamortizationofintangibleliabilitiesfinitelived_qoq',
 'acceleratedamortizationofpatents_qoq',
 'acceleratedcompensationexpense

### The current GAAP_metadata_10Q_Full file contains:
* 79733 Filings 
* 28 columns of metadata for each filing  

In [170]:
metadata_full = load_from_pickle(params.GAAP_metadata_repo_path)
metadata_full

,FileName,AmendmentFlag,CurrentFiscalYearEndDate,DocumentFiscalPeriodFocus,DocumentFiscalYearFocus,EntityCentralIndexKey,DocumentType,DocumentQuarterlyReport,DocumentPeriodEndDate,DocumentTransitionReport,...,CityAreaCode,LocalPhoneNumber,EntityCurrentReportingStatus,EntityInteractiveDataCurrent,EntityFilerCategory,EntitySmallBusiness,EntityEmergingGrowthCompany,EntityShellCompany,TradingSymbol,SecurityExchangeName
0,1593034_0001593034-19-000016,false,--12-31,Q1,2019,0001593034,10-Q,None,2019-03-31,None,...,None,None,None,None,Large Accelerated Filer,false,false,None,ENDP,None
1,315858_0000315858-20-000079,false,--12-31,Q2,2020,0000315858,10-Q,true,2020-06-30,false,...,954,940-4900,Yes,Yes,Accelerated Filer,false,false,false,BBX,NYSE
2,832480_0001140361-21-017387,false,--12-31,Q1,2021,0000832480,10-Q,true,2021-03-31,false,...,217,323-6300,Yes,Yes,Non-accelerated Filer,true,false,false,utgn,None
3,1177702_0001564590-21-021349,false,--12-31,Q1,2021,0001177702,10-Q,true,2021-03-31,false,...,770,232-5067,Yes,Yes,Large Accelerated Filer,false,false,false,SAIA,NASDAQ
4,1769663_0001558370-22-008327,false,--06-30,Q3,2022,0001769663,10-Q,true,2022-03-31,false,...,518,730-3025,Yes,Yes,Non-accelerated Filer,true,true,false,PBFS,NASDAQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79728,1596967_0001564590-20-034909,false,--12-31,Q2,2020,0001596967,10-Q,true,2020-06-30,false,...,212,883-3800,Yes,Yes,Large Accelerated Filer,false,false,false,MC,NYSE
79729,78890_0000078890-22-000426,false,--12-31,Q3,2022,0000078890,10-Q,true,2022-09-30,false,...,804,289-9600,Yes,Yes,Large Accelerated Filer,false,false,false,BCO,NYSE
79730,1463172_0001463172-22-000326,false,--12-31,Q3,2022,0001463172,10-Q,true,2022-09-30,false,...,415,418-7506,Yes,Yes,Large Accelerated Filer,false,false,false,ZEN,NYSE
79731,1284450_0001213900-19-009166,false,--12-31,Q1,2019,0001284450,10-Q,None,2019-03-31,None,...,None,None,None,None,Non-accelerated Filer,true,false,None,CHGI,None


### GAAP_metadata_10Q_Full Column Names:

In [171]:
metadata_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79733 entries, 0 to 79732
Data columns (total 28 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   FileName                             79733 non-null  object
 1   AmendmentFlag                        79733 non-null  object
 2   CurrentFiscalYearEndDate             79733 non-null  object
 3   DocumentFiscalPeriodFocus            79733 non-null  object
 4   DocumentFiscalYearFocus              79732 non-null  object
 5   EntityCentralIndexKey                79733 non-null  object
 6   DocumentType                         79733 non-null  object
 7   DocumentQuarterlyReport              61290 non-null  object
 8   DocumentPeriodEndDate                79733 non-null  object
 9   DocumentTransitionReport             61088 non-null  object
 10  EntityFileNumber                     65203 non-null  object
 11  EntityRegistrantName                 7950